In [1]:
import numpy as np
from collections import Counter

In [2]:
from qulacs import QuantumState, QuantumCircuit
from qulacsvis import circuit_drawer
from scipy.linalg import expm
from qulacs.gate import DenseMatrix
from qulacs.gate import X, Y, Z  , Pauli, Identity, merge
from itertools import combinations
import random
# to simulate time evolution
from qulacs import Observable, PauliOperator, StateVector
# to exponentiate
from scipy.linalg import expm

#
from qulacs.gate import Pauli
from qumcmc.energy_models import IsingEnergyFunction

In [3]:
# function for U=exp(-iHt) (a unitary evolution)
# H =(1-gamma)*alpha*H_{prob} + gamma * H_mix
def time_evol_for_mcmc(num_spins:int, H_prob:np.array, H_mix:np.array, 
                        gamma:float, alpha:float,q_state:QuantumState, time_evol:float):
    target_list=list(range(num_spins-1,0,-1))
    H=(1-gamma)*alpha*H_prob + gamma * H_mix
    unitary_mat= np.round(np.expm(-1j*H*time_evol),decimals=6)
    unitary_time_evol_gate=DenseMatrix(target_list,matrix=unitary_mat)
    # update the quantum state
    unitary_time_evol_gate.update_quantum_state(q_state)
    return q_state

In [4]:
a=QuantumState(2)
a.set_computational_basis(0b11)# i might be required to change this in the original code
a.sampling(sampling_count=10)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
a.set_computational_basis(int('10',2))
print(a)
a.sampling(10)

 *** Quantum State ***
 * Qubit Count : 2
 * Dimension   : 4
 * State vector : 
(0,0)
(0,0)
(1,0)
(0,0)



[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [29]:
op = PauliOperator([1] )

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. qulacs_core.PauliOperator(coef: complex)
    2. qulacs_core.PauliOperator(pauli_string: str, coef: complex)

Invoked with: [1]

In [19]:
op.add_single_Pauli(3,1)

In [21]:
op.get_pauli_string(), op.get_pauli_id_list()

('X 3', [1])

In [25]:
opr = Pauli([1,2,3,],[1,1,1])

In [30]:
coef = 2.0
Pauli_string = "X 0 X 1 Y 2 Z 4"
pauli = PauliOperator ( Pauli_string , coef )


In [32]:
pauli.get_pauli_string()

'X 0 X 1 Y 2 Z 4'

In [ ]:
def create_hamiltonian(data: IsingEnergyFunction):

    J = data.get_J; h = data.get_h
    

### creating mixer hamiltonian

In [3]:
txt= ("X %d "*3) % tuple(range(0,3))
txt

'X 0 X 1 X 2 '

In [5]:
### creating the operator for H_mix:
# H_mix= sum(X_{i}X_{j}...X_{k})
def create_X_mixer_hamiltonian(num_spins:int,weight_individual_pauli:int):
    mixer_hamiltonian=Observable(num_spins)
    print("type_mixer_hamiltonian:",type(mixer_hamiltonian))
    num_sumterms=num_spins-weight_individual_pauli+1
    list_individual_terms=[("X %d "*weight_individual_pauli) % tuple(range(i,i+weight_individual_pauli)) for i in range(0,num_sumterms)]
    print("list_individual_terms:")
    print(list_individual_terms)
    for i in range(0,num_sumterms):
        mixer_hamiltonian.add_operator(coef=1,string=list_individual_terms[i])
    return mixer_hamiltonian

In [8]:
check_mix=create_X_mixer_hamiltonian(3,2)
matrix_mixer=check_mix.get_matrix().toarray()
matrix_mixer

type_mixer_hamiltonian: <class 'qulacs_core.Observable'>
list_individual_terms:
['X 0 X 1 ', 'X 1 X 2 ']


array([[0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [4]:
check_mix=create_X_mixer_hamiltonian(3,2)
print("type:",type(check_mix.get_matrix()))
matrix_mixer=check_mix.get_matrix().toarray()
print("matrix mixer:")
print(matrix_mixer)

type_mixer_hamiltonian: <class 'qulacs_core.Observable'>
list_individual_terms:
['X 0 X 1 ', 'X 1 X 2 ']
type: <class 'scipy.sparse.csr.csr_matrix'>
matrix mixer:
[[0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]]


In [13]:
coef = 2.0
Pauli_string = "X 0 X 1 Y 2 Z 4"
pauli = PauliOperator( Pauli_string , coef )

# Create an observable acting on n qubits
n = 5
observable = Observable( n )
# Add a Pauli operator to the observable
observable.add_operator( pauli )
# or directly add it with coef and str
observable.add_operator(0.5 , "Y 1 Z 4")
observable.get_matrix().A

array([[0.+0.j , 0.+0.j , 0.-0.5j, ..., 0.+0.j , 0.+0.j , 0.+0.j ],
       [0.+0.j , 0.+0.j , 0.+0.j , ..., 0.+0.j , 0.+0.j , 0.+0.j ],
       [0.+0.5j, 0.+0.j , 0.+0.j , ..., 0.+0.j , 0.+0.j , 0.+0.j ],
       ...,
       [0.+0.j , 0.+0.j , 0.+0.j , ..., 0.+0.j , 0.+0.j , 0.+0.5j],
       [0.+0.j , 0.+0.j , 0.+0.j , ..., 0.+0.j , 0.+0.j , 0.+0.j ],
       [0.+0.j , 0.+0.j , 0.+0.j , ..., 0.-0.5j, 0.+0.j , 0.+0.j ]])